# Point Cloud Demonstrations

## Registration

Registers two point clouds with slight offset.


In [1]:
import open3d as o3d
import numpy as np
import copy

import utils

In [ ]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

# load point clouds
base = o3d.io.read_point_cloud("./Point Clouds/Cleaner/1.ply")
source = o3d.io.read_point_cloud("./Point Clouds/Cleaner/8.ply")

# get 4x4 transformation matrix
trans = utils.register_clouds(base, source)

# show original clouds
o3d.visualization.draw_geometries([base, source])

# show aligned clouds
draw_registration_result_original_color(source, base, trans)


## Normals of point cloud
Shows the histogram of normals for the cloud. See utils module - running in notebook is much slower

## Outlier removal
Removes outliers and scatter artifacts from the pointcloud

In [2]:
cl, ind=utils.clean_cloud("./Point Clouds/Cleaner/1.ply", view=True)

## Cleaned and combined 8 clouds

In [ ]:
base = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
o3d.visualization.draw_geometries([base])

## SVD of isolated plane
After isolating one of the planes and performing SVD the vectors aren't what I thought they would be?

In [ ]:
plane = o3d.io.read_point_cloud("./plane.ply")
points = np.asarray(plane.points)
u, s, v = np.linalg.svd(points)

s /= np.linalg.norm(s)

for i in range(len(s)):
    v[i] *= s[i]

o3d.visualization.draw_geometries([plane, utils.create_vector_graph(v)])

## Fitting to plane
Shows normal vector to plane

In [3]:
cl = o3d.io.read_point_cloud("./Point Clouds/combined_cloud_clean.ply")
pcd = cl.voxel_down_sample(0.01)

pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2,
                                                          max_nn=30))
cloud_size = np.asarray(pcd.points).shape[0]

# 1500 is on the plane by guess
pcd.colors[1500] = [1, 0, 0]

# find it's nearest neighbour
# k is number of nearest neighbours, idx is index in list _ is the distances
seeds = {1500}
plane = {1500}
normals = np.asarray(pcd.normals)
match_normal = normals[1500]

# region grow
counter = 0
while seeds != set():
    seed = seeds.pop()
    [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[seed], 30)
    match = np.abs(normals.dot(match_normal))
    plane_addition = {i for i in idx if match[i] > 0.8}
    seed_addition = {i for i in plane_addition if not i in plane}
    seeds.update(seed_addition)
    plane.update(plane_addition)
    
    # get average normal to keep direction fixed
    counter +=1
    counter %= 100
    if counter == 0:
        match_normal = np.array([normals[i] for i in plane]).mean(axis=0)

for i in plane:
    pcd.colors[i] = [1, 0, 0]

plane = pcd.select_down_sample(np.array(list(plane)))

points = np.asarray(plane.points)
pseudoinverse = np.linalg.pinv(points.T)
vec = pseudoinverse.T.dot(np.ones(points.shape[0]))

o3d.visualization.draw_geometries([pcd, utils.create_vector_graph([vec])])

In [4]:
# now match to the original full size point cloud

cl.paint_uniform_color([0.5,0.5,0.5])
dot_match = np.abs(1-np.asarray(cl.points).dot(vec))
for i in range(len(dot_match)):
    if dot_match[i] <0.015:
        cl.colors[i] = [1,0,0]

# add normals for clearer viewing
if not cl.has_normals():
    cl.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=0.04 * 2,
                                             max_nn=30))
o3d.visualization.draw_geometries([cl])